In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pysparkling import *
from pysparkling import *
import h2o

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()
hc = H2OContext.getOrCreate(ss)

Connecting to H2O server at http://192.168.1.13:54323 ... successful.


H2O cluster uptime:,10 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,19 days
H2O cluster name:,sparkling-water-dwoodbridge_local-1578242915878
H2O cluster total nodes:,1
H2O cluster free memory:,808 Mb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.28.0.1-1-2.4
 * H2O name: sparkling-water-dwoodbridge_local-1578242915878
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,192.168.1.13,54323)
  ------------------------

  Open H2O Flow in browser: http://192.168.1.13:54323 (CMD + click in Mac OSX)

    


## Create H2OFrames

In [2]:
adult_train_df = ss.read.parquet("../Day3/spark-warehouse/adulttrain")
adult_train_h2o = hc.as_h2o_frame(adult_train_df, "adult_train")
adult_train_h2o["label"] = adult_train_h2o["label"].asfactor()

adult_valid_df = ss.read.parquet("../Day3/spark-warehouse/adultvalid")
adult_valid_h2o = hc.as_h2o_frame(adult_valid_df, "adult_valid")
adult_valid_h2o["label"] = adult_valid_h2o["label"].asfactor()

## Apply AutoML

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/_modules/h2o/automl/autoh2o.html

http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html#h2o.automl.autoh2o.H2OAutoML


### Identify predictors and response

In [4]:
predictors = adult_valid_h2o.names[:]
response = "label"
predictors.remove(response)

In [38]:
from h2o.automl import H2OAutoML
model_automl = H2OAutoML(max_models = 10, seed = 1, nfolds = 3) # seed : Set a seed for reproducibility
model_automl.train(x=predictors,
                y=response,
                training_frame=adult_train_h2o)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leader Board

Reported on cross-validation data.

In [39]:
model_automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20200105_115745,0.928122,0.29321,0.815897,0.170659,0.300256,0.0901538
StackedEnsemble_BestOfFamily_AutoML_20200105_115745,0.92773,0.293945,0.807327,0.169579,0.300457,0.0902745
GBM_1_AutoML_20200105_115745,0.927386,0.2794,0.817684,0.171176,0.29703,0.0882271
GBM_2_AutoML_20200105_115745,0.927055,0.279891,0.81576,0.170933,0.297476,0.088492
XGBoost_1_AutoML_20200105_115745,0.927044,0.279748,0.814925,0.173098,0.297724,0.0886397
XGBoost_3_AutoML_20200105_115745,0.926826,0.280776,0.814717,0.169161,0.298123,0.0888771
GBM_3_AutoML_20200105_115745,0.9266,0.28078,0.810708,0.172645,0.298027,0.08882
GBM_4_AutoML_20200105_115745,0.926025,0.28185,0.804691,0.162301,0.299011,0.0894074
GBM_5_AutoML_20200105_115745,0.925258,0.282982,0.816486,0.170585,0.299462,0.0896777
XGBoost_2_AutoML_20200105_115745,0.924518,0.284491,0.803384,0.174226,0.300221,0.0901328


### Model Exploration

In [40]:
h2o.get_model("StackedEnsemble_AllModels_AutoML_20200105_115745")

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200105_115745

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.08037051147316902
RMSE: 0.28349693379853164
LogLoss: 0.2634859824339291
Null degrees of freedom: 39013
Residual degrees of freedom: 39006
Null deviance: 42878.5702775286
Residual deviance: 20559.284237354623
AIC: 20575.284237354623
AUC: 0.9443980261207152
AUCPR: 0.8102605093709755
Gini: 0.8887960522414304

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.29710240531089: 


,,0,1,Error,Rate
0,0,26933.0,2770.0,0.0933,(2770.0/29703.0)
1,1,1921.0,7390.0,0.2063,(1921.0/9311.0)
2,Total,28854.0,10160.0,0.1202,(4691.0/39014.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.297102,0.759078,240.0
1,max f2,0.121534,0.831547,320.0
2,max f0point5,0.707480,0.792351,104.0
3,max accuracy,0.455163,0.886502,184.0
4,max precision,0.969009,1.000000,0.0
5,max recall,0.034612,1.000000,398.0
6,max specificity,0.969009,1.000000,0.0
7,max absolute_mcc,0.382168,0.682238,207.0
8,max min_per_class_accuracy,0.202623,0.861765,282.0
9,max mean_per_class_accuracy,0.153433,0.865244,304.0



Gains/Lift Table: Avg response rate: 23.87 %, avg score: 23.92 %


...

,,0,1,Error,Rate
0,0,26733.0,2970.0,0.1,(2970.0/29703.0)
1,1,2247.0,7064.0,0.2413,(2247.0/9311.0)
2,Total,28980.0,10034.0,0.1337,(5217.0/39014.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.302741,0.730318,241.0
1,max f2,0.108410,0.806781,330.0
2,max f0point5,0.691924,0.763285,110.0
3,max accuracy,0.464648,0.875532,184.0
4,max precision,0.969010,1.000000,0.0
5,max recall,0.034196,1.000000,399.0
6,max specificity,0.969010,1.000000,0.0
7,max absolute_mcc,0.383230,0.644695,211.0
8,max min_per_class_accuracy,0.183691,0.841478,292.0
9,max mean_per_class_accuracy,0.145852,0.844270,311.0



Gains/Lift Table: Avg response rate: 23.87 %, avg score: 23.87 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010022,0.968119,4.190098,4.190098,1.000000,0.968591,1.000000,0.968591,0.041993,0.041993,319.009773,319.009773
1,,2,0.020018,0.967400,4.190098,4.190098,1.000000,0.967755,1.000000,0.968174,0.041886,0.083879,319.009773,319.009773
2,,3,0.030015,0.966267,4.190098,4.190098,1.000000,0.966883,1.000000,0.967744,0.041886,0.125765,319.009773,319.009773
3,,4,0.040011,0.964072,4.157866,4.182045,0.992308,0.965248,0.998078,0.967120,0.041564,0.167329,315.786621,318.204502
4,,5,0.050008,0.960074,4.147122,4.175064,0.989744,0.962242,0.996412,0.966145,0.041456,0.208785,314.712237,317.506407
5,,6,0.100015,0.851945,3.638147,3.906606,0.868273,0.911439,0.932342,0.938792,0.181935,0.390721,263.814739,290.660573
6,,7,0.149997,0.704767,2.907283,3.573612,0.693846,0.784910,0.852871,0.887516,0.145312,0.536033,190.728320,257.361206
7,,8,0.200005,0.489172,2.426863,3.286888,0.579190,0.599298,0.784442,0.815452,0.121362,0.657394,142.686337,228.688815
8,,9,0.299995,0.217885,1.587533,2.720485,0.378877,0.331338,0.649265,0.654094,0.158737,0.816131,58.753254,172.048468
9,,10,0.400010,0.099008,0.984705,2.286484,0.235008,0.148874,0.545688,0.527773,0.098486,0.914617,-1.529482,128.648419


In [41]:
h2o.get_model("StackedEnsemble_BestOfFamily_AutoML_20200105_115745")

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20200105_115745

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.07817298805263602
RMSE: 0.27959432764746145
LogLoss: 0.25726137313096803
Null degrees of freedom: 39013
Residual degrees of freedom: 39009
Null deviance: 42878.5702775286
Residual deviance: 20073.590422663176
AIC: 20083.590422663176
AUC: 0.9482005911435538
AUCPR: 0.8154463446538504
Gini: 0.8964011822871076

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3035267635996447: 


,,0,1,Error,Rate
0,0,27067.0,2636.0,0.0887,(2636.0/29703.0)
1,1,1915.0,7396.0,0.2057,(1915.0/9311.0)
2,Total,28982.0,10032.0,0.1167,(4551.0/39014.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.303527,0.764721,240.0
1,max f2,0.116017,0.838304,325.0
2,max f0point5,0.610253,0.798013,140.0
3,max accuracy,0.484535,0.889194,178.0
4,max precision,0.967279,1.000000,0.0
5,max recall,0.036823,1.000000,394.0
6,max specificity,0.967279,1.000000,0.0
7,max absolute_mcc,0.366477,0.689121,218.0
8,max min_per_class_accuracy,0.205929,0.866815,280.0
9,max mean_per_class_accuracy,0.165195,0.870789,299.0



Gains/Lift Table: Avg response rate: 23.87 %, avg score: 23.97 %


...

,,0,1,Error,Rate
0,0,26644.0,3059.0,0.103,(3059.0/29703.0)
1,1,2199.0,7112.0,0.2362,(2199.0/9311.0)
2,Total,28843.0,10171.0,0.1348,(5258.0/39014.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.294507,0.730110,237.0
1,max f2,0.084870,0.806036,344.0
2,max f0point5,0.687005,0.763228,108.0
3,max accuracy,0.509275,0.875378,164.0
4,max precision,0.967449,1.000000,0.0
5,max recall,0.034666,1.000000,399.0
6,max specificity,0.967449,1.000000,0.0
7,max absolute_mcc,0.399617,0.645831,199.0
8,max min_per_class_accuracy,0.183608,0.841048,287.0
9,max mean_per_class_accuracy,0.155193,0.844442,302.0



Gains/Lift Table: Avg response rate: 23.87 %, avg score: 23.87 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010022,0.966479,4.190098,4.190098,1.000000,0.967131,1.000000,0.967131,0.041993,0.041993,319.009773,319.009773
1,,2,0.020018,0.965202,4.190098,4.190098,1.000000,0.965887,1.000000,0.966510,0.041886,0.083879,319.009773,319.009773
2,,3,0.030015,0.963439,4.168610,4.182941,0.994872,0.964329,0.998292,0.965783,0.041671,0.125550,316.861005,318.294129
3,,4,0.040011,0.961195,4.157866,4.176677,0.992308,0.962396,0.996797,0.964937,0.041564,0.167114,315.786621,317.667654
4,,5,0.050008,0.956392,4.136379,4.168621,0.987179,0.959116,0.994874,0.963773,0.041349,0.208463,313.637853,316.862107
5,,6,0.100015,0.850997,3.648886,3.908753,0.870835,0.910450,0.932855,0.937111,0.182472,0.390935,264.888573,290.875340
6,,7,0.149997,0.701483,2.890093,3.569316,0.689744,0.783866,0.851846,0.886047,0.144453,0.535388,189.009305,256.931600
7,,8,0.200005,0.487019,2.444045,3.287962,0.583291,0.595922,0.784698,0.813506,0.122221,0.657609,144.404471,228.796212
8,,9,0.299995,0.217034,1.578940,2.718337,0.376826,0.330368,0.648753,0.652474,0.157878,0.815487,57.893967,171.833665
9,,10,0.400010,0.100500,0.981484,2.284068,0.234239,0.149132,0.545111,0.526622,0.098163,0.913651,-1.851632,128.406776


In [42]:
predictions = model_automl.predict(adult_valid_h2o)
predictions

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
1,0.0322083,0.967792
1,0.0339949,0.966005
0,0.964727,0.035273
1,0.0345517,0.965448
1,0.297851,0.702149
0,0.965633,0.0343674
0,0.964709,0.0352914
0,0.962641,0.037359
0,0.9652,0.0347997
1,0.310342,0.689658
